<div style='float: right'><img src='pic/wall.png'/></div>
## <div id='wall' />ウォールロジック

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
4..1..
.4..2.
..2..2
1..1..
.1..1.
..3..2""".split()
nw, nh = len(data[0]), len(data)
mx = max(nw, nh)

### 問題
* 数字が記入されているマスからその数字の数だけ縦と横に線を引きます
* 1つのマスには1本の線しか引くことができません
* 数字が記入されているマスには線を引くことができません

### 変数
* v：各位置のどの方向か  (1)
* r：各位置の方向別長さ  (2)

### 制約
* 数字があれば、方向別長さの和に等しく、かつその位置に矢印がないこと  (3)
* 数字がなければ矢印は1方向のみ  (4)
* 数字がなければ矢印の方向に長さを1足すこと  (5)

In [ ]:
m = LpProblem()
v = np.array(addbinvars(nh, nw, 4)) # (1)
r = np.array(addvars(nh, nw, 4)) # (2)
drc = [(-1, 0, 0), (0, -1, 1), (0, 1, 2), (1, 0, 3)]
for i, j in product(range(nh), range(nw)):
    if data[i][j].isdigit():
        m += lpSum(r[i+y,j+x,k] for y,x,k in drc
                   if 0<=i+y<nh and 0<=j+x<nw) == int(data[i][j]) # (3)
        m += lpSum(r[i,j]) == 0 # (3)
        continue
    m += lpSum(v[i,j]) == 1 # (4)
    for y, x, k in drc:
        m += r[i,j,k] <= mx * v[i,j,k] # (5)
        m += r[i,j,k] <= v[i,j,k] + (r[i+y,j+x,k]
                         if 0<=i+y<nh and 0<=j+x<nw else 0) # (5)
%time m.solve()
print('\n'.join(''.join(' '+data[i][j] if data[i][j].isdigit() else '↑←→↓'
  [int(value(lpDot(np.arange(4),v[i,j])))] for j in range(nw)) for i in range(nh)))